<a href="https://colab.research.google.com/github/skauntey/movie-sentiment-analysis/blob/ALMS/All_words_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir ~/.kaggle
!echo '{"username":"kaunteyshah","key":"e2678a9d0e44e057000b9daf059111b5"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, shutil
import kaggle

#### 1. Downloading Kaggle files

In [ ]:
kaggle.api.authenticate()
kaggle.api.competition_download_files('sentiment-analysis-on-movie-reviews', path= str(os.getcwd())+"/dataset/", force = True)

#### 2. Unzipping Kaggle files

In [ ]:
import zipfile
ziped_file = r'dataset/sentiment-analysis-on-movie-reviews.zip'
folder_path = os.path.join(str(os.getcwd()), 'dataset/')
for files in os.listdir(folder_path):
  path = os.path.join(os.getcwd()+'/'+'dataset/',files)
  if path.split('.')[-1] != "zip":
    try:
      shutil.rmtree(path)
    except:
      raise
with zipfile.ZipFile(ziped_file) as zip_file:
  for member in zip_file.namelist():
      if member.split('.')[-1] == "zip":
          fdir = member.split('.')[0]
          zip_file.extract(member, path= os.path.join('dataset/'+fdir))
          # extracting individual train and test files in their respective folders
          zippedfile_name = os.listdir(os.path.join(str(os.getcwd()+'/dataset/'+fdir+'/')))
          zippedfile = os.path.join(str(os.getcwd())+'/dataset/'+ fdir, zippedfile_name[0])
          with zipfile.ZipFile(zippedfile, mode='r') as tsv_zip:
              tsv_zip.extractall(path = os.path.join('dataset/'+fdir+'/'))
                
          os.remove(zippedfile)
      else:
        continue
            
os.remove(ziped_file)


#### 3. Testing files

In [ ]:
tsvfile = 'dataset/train/train.tsv'
# read the data
tsv_read = pd.read_csv(tsvfile, sep="\t")

original_dataset = pd.DataFrame({ 
    'DATA_COLUMN': [tsv_read.loc[:,'Phrase'][x].lower() for x in range(len(tsv_read))], 
    'LABEL_COLUMN': np.array([tsv_read.loc[:,'Sentiment'][x] for x in range(len(tsv_read))], dtype='int32')},
    columns= ['DATA_COLUMN','LABEL_COLUMN'])

pdf_data = original_dataset['DATA_COLUMN']
print(original_dataset['DATA_COLUMN'].head())
print(pdf_data.head())
pdf_labels = original_dataset['LABEL_COLUMN']
print(type(pdf_data))

In [ ]:
train_split = 0.8
val_split = 0.2
test_split = 0

training_mark = int(abs(len(original_dataset) * train_split))
# validation_mark = np.add(training_mark, int(abs(len(pdf)) * val_split))

print(training_mark)
# print(validation_mark)

training_set = original_dataset.loc[:training_mark]
train_data = training_set['DATA_COLUMN']
train_labels = training_set['LABEL_COLUMN']
validation_set = original_dataset.loc[training_mark:]
val_data = validation_set['DATA_COLUMN']
val_labels = validation_set['LABEL_COLUMN']
# pdf.loc[training_mark:validation_mark]
# test_set = np.array(pdf.loc[validation_mark:])

print(train_labels.head())
print(val_labels.head())
print(type(val_labels))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
"""stopwords = [ "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "into", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

def remove_stopwords(dataseries, original_dataset):
  for word in stopwords:
    tokens = [" "+word+" ", " "+word, word+" "]
    for token in tokens:
      dataseries = [dataseries[i].replace(token, " ") for i in range(len(dataseries))]
      original_dataset['DATA_COLUMN'] = dataseries
      new_pdf_data = original_dataset['DATA_COLUMN']
  return dataseries, new_pdf_data

_, new_pdf_data = remove_stopwords(pdf_data, original_dataset)

print(new_pdf_data[0])"""


In [ ]:

def tokenizing_dataset(original_dataset):
  tokenizer = Tokenizer(num_words = 16000, oov_token='<00V>')
  tokenizer.fit_on_texts(original_dataset)
  word_index = tokenizer.word_index
  return word_index, tokenizer

def Sequencing_dataset(original_dataset, dataset):
  _, pretrained_tokenizer = tokenizing_dataset(original_dataset)
  sequences = pretrained_tokenizer.texts_to_sequences(dataset)
  padded_sentences = pad_sequences(sequences, maxlen= 50, truncating ='post', padding='post')
  return padded_sentences, sequences

padded_train, train_sequences = Sequencing_dataset(pdf_data, train_data)
padded_val, val_sequences = Sequencing_dataset(pdf_data, val_data)
print(max([len(x) for x in val_sequences]))
print(padded_val[0])
print(val_data[124848])
print(padded_train[0])
print(train_data[0])

In [ ]:
def decoding_sequences(original_dataset, sequences):
  word_index, _ = tokenizing_dataset(original_dataset)
  reversed_index = dict(map(reversed, word_index.items()))
  for letter in range(len(sequences)):
      reversed_index.get(letter) 
  reversed_sequence = [reversed_index.get(letter) for letter in sequences]
  return (' '.join(reversed_sequence))
    
print(decoding_sequences(pdf_data, train_sequences[0]))
print(train_data[0])

In [ ]:
vocabulary_size = 16000
embedding_dim = 128
max_length = 50
trunc_type = 'post'
oov_tok = '<00V>'

model = tf.keras.Sequential([
                             
    tf.keras.layers.Embedding(vocabulary_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(5, activation ='sigmoid')
])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), metrics= ['accuracy'])
model.summary()

In [ ]:
num_epochs = 20

history = model.fit(padded_train, train_labels, batch_size=10000, epochs = num_epochs, validation_data = (padded_val, val_labels))

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)